# Taller - Examen

## Punto 1:

Un pequeño aeropuerto necesita organizar el horario de los vuelos de un día. El aeropuerto puede aceptar 10 vuelos cada hora y esta abierto 15 horas al día. Para establecer el horario de vuelos, el aeropuerto puede escoger vuelos de 10 compañías diferentes y cada compañía ofrece vuelos a 10 destinos diferentes.

El horario no puede tener más de dos vuelos de la misma compañía a la misma hora y dos vuelos hacia el mismo destino de una compañía han de estar separados al menos seis horas.

La ganancia que obtiene el aeropuerto por tener un vuelo de una compañía a un destino depende de su demanda. Queremos maximizar la ganancia del aeropuerto cumpliendo las restricciones. Podemos resolver el problema de dos formas diferentes, de las cuales especificaremos una de ella:

* Usando A*, definiendo como estado la asignación de vuelos al horario. El estado inicial es el horario vacío, el estado final se obtiene cuando el horario está lleno. Como operador de búsqueda usamos la asignación de un vuelo de una compañía a un destino a una hora específica del horario, el coste será el dinero que el aeropuerto gana con este vuelo. Como función heurística usamos el número de vuelos que aún se han de asignar al estado.

Comenta esta posibilidad indicando si resuelven o no el problema, qué errores te parece que tiene la solución y cómo se podrían corregir, y qué ventajas e inconvenientes tiene. Justifica la respuesta.

---
---

### JUAN MANUEL AGUDELO AGUIRRE - 1004682681


# RESPUESTA 
### ORGANIZACION DE VUELOS
1. Representación del problema

    - Estado: Una matriz de 15×10 donde cada celda representa una hora y contiene una lista de vuelos asignados.

    - Operadores: Agregar un vuelo de una compañía a un horario específico.

    - Restricciones:
        - No más de dos vuelos de la misma compañía a la misma hora.
        - Vuelos al mismo destino de una compañía separados al menos 6 horas.

    - Función de costo: Ganancia negativa del aeropuerto para que A* minimice el costo.

    - Función heurística: Número de vuelos restantes penalizado por violaciones de restricciones.

2. Diseño del algoritmo

    - Definir el espacio de estados:
        - Crear una estructura que represente el horario parcial.
        - Incorporar una lista de vuelos pendientes para asignar.

    - Crear operadores válidos:
        - Verificar restricciones antes de aplicar el operador.

    - Implementar una función heurística mejorada:
        - Penalizar estados con conflictos.
        - Priorizar estados que maximicen la ganancia.

    - Estructurar el algoritmo A*:
        - Usar una cola de prioridad basada en f(n)=g(n)+h(n)f(n)=g(n)+h(n).
        - Explorar estados en orden de costo estimado más bajo.

*Entrada*: Lista de vuelos disponibles (compañía, destino, ganancia).

*Salida*: Horario óptimo que maximiza la ganancia cumpliendo restricciones.

*Complejidad*: Depende del número de vuelos y restricciones, pero se mitiga con poda y heurística.

--- 

In [ ]:
from collections import defaultdict;
# Definir el espacio de estados
class EstadoHorario:
    def __init__(self):
        # Matris de 15 x 10
        self.horario = [[[] for _ in range(10)] for _ in range(15)]

        # Verificamos vuelos pendientes
        self.vuelosPendientes = defaultdict(list) # Estructura del diccionario {compañia: [(destino,ganancia)]}

    def restricciones(self,hora,posicion,compania, destino):
        # Verifica que no haya mas de dos vuelos de la misma compañia en la misma hora 
        vuelosHora = [vuelo[0] for vuelo in self.horario[hora]]
        if vuelosHora.count(compania) >=2:
            return False
        
        # Verifica que exista una separacion de 6 horas entre vuelos al mismo destino
        for h in range(max(0, hora-6), min(15,hora-6)):
            if any(v[1] == destino and v[0] == compania for v in self.horario[h]):
                return False
        
        return True

---

In [ ]:
# Crear operadores validos

# Operadores: asignan vuelos al horario si cumplen las restricciones
def asignarVuelo(estado, hora, posicion, compania, destino, ganancia):
    """
    Asigna un vuelo al horario si es válido.
    """
    if estado.es_valido(hora, posicion, compania, destino):
        nuevoEstado = EstadoHorario()
        nuevoEstado.horario = [fila[:] for fila in estado.horario]
        nuevoEstado.horario[hora][posicion] = (compania, destino, ganancia)
        nuevoEstado.vuelosPendientes = estado.vuelosPendientes.copy()
        nuevoEstado.vuelosPendientes[compania].remove((destino, ganancia))
        return nuevoEstado
    
    return None

---

In [ ]:
#Implementar heurística mejorada
def heuristica(estado):
    """
    Calcula la heurística como el número de vuelos pendientes menos una penalización por restricciones.
    """
    penalizacion = 0
    for hora in range(15):
        for posicion in range(10):
            vuelos = estado.horario[hora][posicion]
            if len(vuelos) > 1:
                penalizacion += 10  # Penalización por conflicto
    vuelosRestantes = sum(len(v) for v in estado.vuelosPendientes.values())
    
    return vuelosRestantes + penalizacion

---

In [ ]:
import heapq;
# Algoritmo A*: Para explorar los estados

def aStar(InitialState):
    openList = []
    heapq.heappush(openList,(0,InitialState)); #(costo,estado)
    closedList = set()

    while openList:
        actualCost,actualState = heapq.heappop(openList)

        # Verifica si el estado esta completo
        if all(not vuelos for vuelos in actualState.vuelosPendientes.values()):
            return actualState
        
        closedList.add(str(actualState.horario)) # hash del estado


        # genera los sucesores
        for hora in range(15):
            for posicion in range(10):
                for compania, vuelos in actualState.vuelosPendientes.items():
                    newState = asignarVuelo(actualState,hora,posicion,compania,destino, ganancia)
                    if newState and str(newState.horario) not in closedList:
                            newCost = actualCost + (-ganancia) + heuristica(newState)
                            heapq.heappush(openList, (newCost, newState))


---

## Punto 2:

Después de los incendios del verano se nos ha planteado rediseñar la ubicación de las estaciones de bomberos. Para solucionar el problema dividimos el área en una cuadrícula de $N×M$, cada posición tiene asignado un factor de accesibilidad A que toma un valor entero entre 1 y 3 (1 es una zona de fácil acceso, 3 es un área de difícil acceso).

Deseamos ubicar un total de P estaciones de bomberos, cada estación puede tener entre uno y tres camiones. Tenemos un total de C camiones para repartir (obviamente C > P ).

Definimos el factor de seguridad de una posición como la suma para esa posición y todas las que la rodean del cociente entre el número de camiones de bomberos que hay en esa posición y el factor de accesibilidad de cada posición.

El objetivo es ubicar todos las estaciones de bomberos y repartir entre ellos todos los camiones de manera que el factor de seguridad global (la suma para todas las posiciones) sea el máximo posible.

* Queremos utilizar A* de manera que recorremos la cuadrícula de la esquina superior izquierda a la inferior derecha. El estado es la asignación que hemos hecho de estaciones de bomberos y camiones a las áreas recorridas. Utilizamos como operador poner una estación de bomberos, asignando uno, dos o tres camiones en el caso de ponerlo, el coste del operador es el número de camiones asignados más uno, o no ponerlo con coste uno. La función heurística es el número de áreas que nos quedan por visitar y vale infinito si ya hemos asignado más de C camiones o P estaciones de bomberos.

# RESPUESTA

### ESTACIONES DE BOMBEROS
1. Representación del problema
    - Estado: Una cuadrícula N×MN×M donde cada celda contiene:

        - Número de estaciones de bomberos.
        - Número de camiones asignados.

    - Restricciones:

        - No exceder PP estaciones de bomberos.
        - Asignar exactamente CC camiones.
        - Respetar el rango de 1 a 3 camiones por estación.

    - Función de costo: 
        - Penalizar la cantidad de camiones y estaciones usadas para minimizar costos.

    - Función heurística: 
        - Aproximar la mejora en el factor de seguridad con los camiones restantes.

2. Diseño del algoritmo
    - Estado inicial: 
        - Cuadrícula vacía sin estaciones ni camiones asignados.

    - Espacio de estados:

        -  Asignar estaciones de bomberos y camiones progresivamente.

    - Operadores válidos:

        -  Colocar una estación en una celda.
        - Asignar 1, 2 o 3 camiones si la celda tiene una estación.

    - Función heurística:

        - Calcular el potencial incremento en el factor de seguridad basado en las celdas restantes.

    - Criterio de finalización:

        - Todas las estaciones (PP) y camiones (CC) han sido asignados.

#### Ventajas del enfoque

    *Restricciones respetadas*: Las restricciones son verificadas en cada paso.

    *Óptimo global*: A* asegura la solución con el mayor factor de seguridad.

    *Escalabilidad*: Puede adaptarse a cuadrículas más grandes y restricciones adicionales.

#### Limitaciones

    *Complejidad computacional*: Puede ser costoso si N×MN×M es grande.

    *Dependencia de la heurística:* Una mala heurística puede ralentizar la solución.

---

In [ ]:
# Representación del estado: cada celda almacena los datos necesarios
class EstadoBomberos:
    def __init__(self, n, m, p, c):
        self.grid = [[{'estacion': 0, 'camiones': 0} for _ in range(m)] for _ in range(n)]
        self.pRestantes = p  # Estaciones restantes por asignar
        self.cRestantes = c  # Camiones restantes por asignar
        self.n = n
        self.m = m
    
    def valido(self, x, y, camiones):
        """
        Verifica si es válido colocar una estación y asignar camiones en una celda.
        """
        if self.grid[x][y]['estacion'] == 1:
            return False  # Ya hay una estación en esta celda
        if camiones > self.cRestantes:
            return False  # No quedan suficientes camiones
        return True


---

In [ ]:
# Operadores: definir estaciones y asignar camiones 
def asignarEstacion(estado, x, y, camiones):
    """
    Asigna una estación de bomberos con camiones a una celda específica.
    """
    if estado.valido(x, y, camiones):
        newState = EstadoBomberos(estado.n, estado.m, estado.pRestantes, estado.cRestantes)
        # Copia del estado actual
        newState.grid = [fila[:] for fila in estado.grid]
        newState.pRestantes = estado.pRestantes - 1
        newState.cRestantes = estado.cRestantes - camiones
        
        # Asignar estación y camiones
        newState.grid[x][y]['estacion'] = 1
        newState.grid[x][y]['camiones'] = camiones
        return  newState
    return None


---

In [ ]:
# Heurística: estima la mejora potencial en factor de seguridad global

def heuristica(estado):
    """
    Calcula una heurística basada en el potencial incremento de seguridad.
    """
    incrementoPotencial = 0
    for x in range(estado.n):
        for y in range(estado.m):
            if estado.grid[x][y]['estacion'] == 0:  # Solo considerar celdas vacías
                incrementoPotencial += 3 / 1  # Máxima mejora posible en áreas difíciles
    return estado.pRestantes + estado.cRestantes - incrementoPotencial



---

In [ ]:
# Algoritmo A* para explorar los  estados
import heapq

def aStarBomberos(initialState):
    """
    Implementa el algoritmo A* para maximizar el factor de seguridad global.
    """
    openList = []
    heapq.heappush(openList, (0, initialState))  # (costo, estado)
    closedList = set()

    while openList:
        actualCost, actualState = heapq.heappop(openList)

        # Si no quedan estaciones ni camiones, terminamos
        if actualState.pRestantes == 0 and actualState.cRestantes == 0:
            return actualState

        closedList.add(str(actualState.grid))  # Hash del estado

        # Generar sucesores
        for x in range(actualState.n):
            for y in range(actualState.m):
                for camiones in range(1, 4):
                    newState = asignarEstacion(actualState, x, y, camiones)
                    if newState and str(newState.grid) not in closedList:
                        costo_nuevo = actualCost + camiones + 1 + heuristica(newState)
                        heapq.heappush(openList, (costo_nuevo, newState))
    return None


---

In [ ]:
# ejemplo de uso: cuadricua 3x3 con P=2 y C=4 

# Crear estado inicial
n, m, p, c = 3, 3, 2, 4
initialState = EstadoBomberos(n, m, p, c)

# Ejecutar A*
resultado = aStarBomberos(initialState)

# Imprimir el resultado
if resultado:
    for fila in resultado.grid:
        print(fila)
else:
    print("No se encontró una solución.")

